## Минимальный пример использования lightgbm чтобы не переписывать каждый раз один и тот же код и вспоминать какие бывают параметры, этот пример не является инструкцией о том как правильно строить модели

In [3]:
import numpy as np
import pandas as pd
import lightgbm as lgb

In [4]:
from turbodbc import connect
connection = connect(dsn='Teradata')

In [51]:
query = "SELECT TOP 100000 SUBS_ID, MSISDN, FLASH_CODE, EVENT_DATE, target_var, bal_avg_li_days,bal_avg_ld_days,lifetime,bc_lifetime,last_flash,bc_bal_avg_li_days,cpe_p_age,SMS_900_days,days_with_data_trafic,cl_avg_lifetime,sum_ses_mou,avg_day_mou,bc_bal_avg_ld_days,bal_min_amt,SMS_900_cnt,sk_day_mou,internet_ap_blocks_cnt,ret_dev_ind,bc_avg_day_voice_cnt,cl_tail_size FROM UAT_DM.AR_CREDIT_LIMIT_PROMISE_PAYMENT_v2_DMSC WHERE SUBS_ID IN (SELECT SUBS_ID FROM UAT_DM.AR_CREDIT_LIMIT_SAMPLE_SUBS_FOR_BALANCE)"

In [52]:
cursor = connection.cursor()

In [53]:
cursor.execute(query)
dt = pd.DataFrame(cursor.fetchallnumpy())

In [54]:
features = dt.columns[~dt.columns.isin(['SUBS_ID', 'MSISDN', 'FLASH_CODE', 'EVENT_DATE', 'target_var'])]
#categorical_features = ['CPE_MANUFACTURER_NAME']

In [105]:
train = lgb.Dataset(data = dt[features].values, label=dt['target_var'].values,
                          feature_name='auto'
                          )

In [100]:
?lgb.Dataset

In [106]:
train.construct()

In [107]:
params = {
    'learning_rate': 0.15,
    #'is_unbalance': 'true', # replaced with scale_pos_weight argument
    'num_leaves': 7,  # 2^max_depth - 1
    'max_depth': 3,  # -1 means no limit
    'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
    'max_bin': 100,  # Number of bucketed bin for feature values
    'subsample': 0.7,  # Subsample ratio of the training instance.
    'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
    'colsample_bytree': 0.9,  # Subsample ratio of columns when constructing each tree.
    'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
    'scale_pos_weight':0.01 # because training data is extremely unbalanced 
}

In [108]:
# lgb_model = lgb.train(params, 
#                  dtrain, 
#                  valid_sets=[dtrain, dvalid], 
#                  valid_names=['train','valid'], 
#                  evals_result=evals_results, 
#                  num_boost_round=350,
#                  early_stopping_rounds=30,
#                  verbose_eval=True, 
#                  feval=None)

evals_results = {}
lgb_model = lgb.train(params, 
                 train,  
                 evals_result=evals_results, 
                 num_boost_round=20)

In [109]:
print('Feature importances:', list(lgb_model.feature_importance()))

Feature importances: [11, 14, 18, 2, 15, 7, 7, 2, 2, 11, 5, 0, 5, 7, 2, 4, 0, 0, 4, 4]


In [ ]:
nrounds = 20
booster = xgb.train(params, dm, nrounds)

In [ ]:
booster.save_model('xgb_example.xgb')

In [ ]:
booster_2 = xgb.Booster()
booster_2.load_model('xgb_example.xgb')